Reference: https://jakevdp.github.io/blog/2013/08/28/understanding-the-fft/

In [2]:
import numpy as np
import math
import cmath
import random
import pyfftw
np.set_printoptions(4)

In [3]:
def DFTUsingForLoops(xin):
    N = xin.shape[0]
    Xout = np.zeros(xin.shape, dtype=np.complex)
    for k in range(N):
        tempX = 0
        for n in range(N):
            expVal = k * (2 * math.pi / N) * n
            expValWithJ = cmath.exp(-1j*expVal)
            tempX = tempX + (xin[n]*expValWithJ)
        Xout[k] = tempX
    return Xout   

In [4]:
def DFTUsingVectorProduct(xin):
    N = xin.shape[0]
    kVector = np.arange(0, N, 1)
    nVector = kVector.reshape(-1, 1)
    ExpMat = np.exp((-1j*2*np.pi/N)*kVector* nVector)
    return np.dot(xin, ExpMat)   

In [5]:
def DFT2Point(xin):
    return np.asarray([xin[0]+xin[1], xin[0]-xin[1]])


def DFTUsingDITRecursive_2powers(xin):
    N = xin.shape[0]
    if np.log2(N) % 1 > 0:
        print('N must powers of 2')
        return
    # if N <= 32:
    #     return DFTUsingVectorProduct(xin)
    if N == 2:
        return DFT2Point(xin)
    
    xeven = xin[0::2]
    xodd = xin[1::2]

    Xeven = DFTUsingDITRecursive_2powers(xeven)
    Xodd = DFTUsingDITRecursive_2powers(xodd)

    WN = np.exp(-1j * (2*np.pi/N) * np.arange(0, N))

    XoddM = np.multiply(Xodd, WN[ : N//2])

    Temp1 = Xeven + XoddM
    Temp2 = Xeven - XoddM
    return np.hstack((Temp1, Temp2))

In [6]:
print(DFTUsingVectorProduct(np.asarray([-2, 7, -1, 6])))
print(DFTUsingDITRecursive_2powers(np.asarray([-2, 7, -1, 6])))

[ 10.+0.0000e+00j  -1.-1.0000e+00j -16.-3.3065e-15j  -1.+1.0000e+00j]
[ 10.+0.j  -1.-1.j -16.+0.j  -1.+1.j]


In [7]:
# np.random.seed(15)
tempX = np.random.uniform(-1,1,16)
print(tempX)


[-0.2159 -0.1279 -0.3958 -0.9057 -0.9885 -0.0409 -0.295  -0.6527 -0.3041
  0.9768 -0.1928 -0.1036 -0.8833  0.4259  0.2012 -0.7763]


In [8]:
gg =DFTUsingDITRecursive_2powers(tempX)
print(gg)

[-4.2787+0.j     -0.9676+2.1474j  1.3831-0.1301j -0.2794+0.81j
 -1.7095-3.6722j  0.0411+0.0316j  1.3205-1.1196j  1.5587+0.9481j
 -1.8698+0.j      1.5587-0.9481j  1.3205+1.1196j  0.0411-0.0316j
 -1.7095+3.6722j -0.2794-0.81j    1.3831+0.1301j -0.9676-2.1474j]


In [9]:
print(DFTUsingForLoops(tempX))
print(DFTUsingVectorProduct(tempX))
print(DFTUsingDITRecursive_2powers(tempX))
print(np.fft.fft(tempX))

[-4.2787+0.0000e+00j -0.9676+2.1474e+00j  1.3831-1.3008e-01j
 -0.2794+8.1000e-01j -1.7095-3.6722e+00j  0.0411+3.1636e-02j
  1.3205-1.1196e+00j  1.5587+9.4811e-01j -1.8698+3.0980e-15j
  1.5587-9.4811e-01j  1.3205+1.1196e+00j  0.0411-3.1636e-02j
 -1.7095+3.6722e+00j -0.2794-8.1000e-01j  1.3831+1.3008e-01j
 -0.9676-2.1474e+00j]
[-4.2787+0.0000e+00j -0.9676+2.1474e+00j  1.3831-1.3008e-01j
 -0.2794+8.1000e-01j -1.7095-3.6722e+00j  0.0411+3.1636e-02j
  1.3205-1.1196e+00j  1.5587+9.4811e-01j -1.8698+3.0980e-15j
  1.5587-9.4811e-01j  1.3205+1.1196e+00j  0.0411-3.1636e-02j
 -1.7095+3.6722e+00j -0.2794-8.1000e-01j  1.3831+1.3008e-01j
 -0.9676-2.1474e+00j]
[-4.2787+0.j     -0.9676+2.1474j  1.3831-0.1301j -0.2794+0.81j
 -1.7095-3.6722j  0.0411+0.0316j  1.3205-1.1196j  1.5587+0.9481j
 -1.8698+0.j      1.5587-0.9481j  1.3205+1.1196j  0.0411-0.0316j
 -1.7095+3.6722j -0.2794-0.81j    1.3831+0.1301j -0.9676-2.1474j]
[-4.2787+0.j     -0.9676+2.1474j  1.3831-0.1301j -0.2794+0.81j
 -1.7095-3.6722j  0.0411

In [10]:
x1 = np.random.random(1024)

In [11]:
# %timeit DFTUsingForLoops(x1)

In [12]:
# %timeit DFTUsingVectorProduct(x1)

In [13]:
%timeit DFTUsingDITRecursive_2powers(x1)

24.9 ms ± 1.86 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%timeit np.fft.fft(x1)
%timeit pyfftw.interfaces.numpy_fft.fft(x1)

29.1 µs ± 4.48 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
116 µs ± 6.05 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [15]:
xin = np.random.random(1024)
# print(xin)
ans = DFTUsingDITRecursive_2powers(xin)
ans2 = np.fft.fft(xin)
print(np.max(np.abs(ans-ans2)))

1.2560739669470201e-14
